In [311]:
import time
import re
start_time = time.time()

import pandas as pd

In [312]:
MMSTA_path = r"C:\Users\user\Desktop\Data integration PP\ALPINE PPC DG\MMSTA AVM ALPINE PPC DG.XLSX"

In [313]:
df_mmsta = pd.read_excel(MMSTA_path)
df_mmsta

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status,Follow-Up Material
0,0,MA13,YFG,F000300478,403881526S,7041118.0,2,NaN,1,0,...,NaN,EX,0,0,NaN,L,NaN,0,NaN,NaN
1,*1,MA13,YSFG,S009238422,Super Group 001 See drawing,7041109.0,2,NaN,1,0,...,NaN,YX,0,25,NaN,L,PC,1,NaN,NaN
2,**2,MA13,YSFG,S009238343,Double Crimping 001 See drawing,7041041.0,2,NaN,2,0,...,NaN,YX,0,25,NaN,L,PC,1,NaN,NaN
3,***3,MA13,YSFG,S009238020,Circuit 0258 COFLRYB 5 R,7040718.0,92,NaN,2,0,...,NaN,YX,0,25,M,L,PC,1,NaN,NaN
4,****4,MA13,YDM,FW404450,LT Single Wire COFLRYB 5 R,NaN,92,NaN,0,0,...,NaN,ZK,0,1500,NaN,L,MM,1390,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11705,**2,MA13,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,NaN,92,NaN,0,0,...,NaN,ZK,0,14000,NaN,L,MM,465,NaN,NaN
11706,*1,MA13,YSFG,S009238175,Circuit 0014 COFLRYB 0.35 P/W,7040873.0,92,NaN,2,0,...,NaN,YX,0,25,NaN,L,PC,1,NaN,NaN
11707,**2,MA13,YDM,7195028702,Terminal 025 Male 0.64mm Unsealed,NaN,92,NaN,0,0,...,NaN,ZK,0,60000,R,L,PC,1,NaN,NaN
11708,**2,MA13,YDM,7196120602,Terminal 025 Female 0.64mm Unsealed,NaN,92,NaN,0,0,...,NaN,ZK,0,48000,L,L,PC,1,NaN,NaN


In [314]:
df_mmsta = df_mmsta[['Level', 'Material Type' , 'Material', 'Material Description', 'Component quantity']]

### Add Columns 'PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3'

In [315]:
def add_columns(df):
    global cols_to_add 
    cols_to_add = ['PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3']
    col_index = df.columns.get_loc('Material Description') + 1
    for i, col in enumerate(cols_to_add):
        df.insert(col_index + i, col, '')
    return df
df_mmsta = add_columns(df_mmsta)
df_mmsta

,Level,Material Type,Material,Material Description,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,0,YFG,F000300478,403881526S,,,,,,,,0
1,*1,YSFG,S009238422,Super Group 001 See drawing,,,,,,,,1
2,**2,YSFG,S009238343,Double Crimping 001 See drawing,,,,,,,,1
3,***3,YSFG,S009238020,Circuit 0258 COFLRYB 5 R,,,,,,,,1
4,****4,YDM,FW404450,LT Single Wire COFLRYB 5 R,,,,,,,,1390
...,...,...,...,...,...,...,...,...,...,...,...,...
11705,**2,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,,,,,,,,465
11706,*1,YSFG,S009238175,Circuit 0014 COFLRYB 0.35 P/W,,,,,,,,1
11707,**2,YDM,7195028702,Terminal 025 Male 0.64mm Unsealed,,,,,,,,1
11708,**2,YDM,7196120602,Terminal 025 Female 0.64mm Unsealed,,,,,,,,1


### Fill PN(Part Number)

In [316]:
def Fill(level : str):
    for idx in range(df_mmsta.shape[0]):
        if df_mmsta.loc[:,'Level'][idx] == level:
            # PN_arr.append(df_mmsta.loc[idx]['Material Description'])
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx]['Material Description']
            idx_last_l = idx
        else :
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx_last_l]['Material Description']
            # PN_arr.append(df_mmsta.loc[idx_last_l]['Material Description'])
    return df_mmsta

In [317]:
df_mmsta = Fill('0')

### Delete YFG And YDM (column = 'Material Type')

In [318]:
def delete_YFG_YDM():
    return df_mmsta[~df_mmsta['Material Type'].isin(['YFG', 'YDM'])]
df_mmsta = delete_YFG_YDM()

### Processing Material Description column (del Cut Tube , Gaft...)

In [288]:
# def del_vals(mat_desc : str):
#     vals_to_del = ['ctgf', 'gaft', 'cut tube']
#     for v in vals_to_del:
#         if v in mat_desc.lower():
#             return False
#     return True

# proccessed_log = df_mmsta['Material Description'].apply(del_vals)

# df_mmsta = df_mmsta[proccessed_log].reset_index(drop=True)

In [319]:
df_mmsta.reset_index(drop=True, inplace=True)
df_mmsta

,Level,Material Type,Material,Material Description,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,*1,YSFG,S009238422,Super Group 001 See drawing,403881526S,,,,,,,1
1,**2,YSFG,S009238343,Double Crimping 001 See drawing,403881526S,,,,,,,1
2,***3,YSFG,S009238020,Circuit 0258 COFLRYB 5 R,403881526S,,,,,,,1
3,***3,YSFG,S009238019,Circuit 0248 COFLRYB 3 R,403881526S,,,,,,,1
4,**2,YSFG,S009238342,Joint 0618 See drawing,403881526S,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3335,**2,YSFG,S009238433,Circuit 0339 COFLRYB 0.5 W,403885584S,,,,,,,1
3336,*1,YSFG,S009238172,Circuit 0005 COFLRYB 0.35 G/W,403885584S,,,,,,,1
3337,*1,YSFG,S009238173,Circuit 0011 COFLRYB 0.35 P/W,403885584S,,,,,,,1
3338,*1,YSFG,S009238174,Circuit 0013 COFLRYB 0.35 L/W,403885584S,,,,,,,1


In [320]:
df_mmsta['Level'].unique()

array(['*1', '**2', '***3'], dtype=object)

In [321]:
def Fill_with_col(level : str, cols,df):
    idx_last_l = None
    for idx in range(df.shape[0]):
        if df.loc[idx,'Level'] == level:
            df.loc[idx,cols[0]] = df.loc[idx,'Material']
            df.loc[idx,cols[1]] = df.loc[idx,'Material Description']
            idx_last_l = idx
           
            
        else :
            if idx_last_l is not None:
                df.loc[idx,cols[0]] = df.loc[idx_last_l,'Material']
                df.loc[idx,cols[1]] = df.loc[idx_last_l,'Material Description']
                
            else :
                df.loc[idx,cols[0]] = '(Blank)'
                df.loc[idx,cols[1]] = '(Blank)'
    return df

for level in df_mmsta['Level'].unique().tolist():
    sn = 'SN'+ level[-1]
    des = 'DES' + level[-1]
    df_mmsta = Fill_with_col(level, [sn, des], df_mmsta)
# df_mmsta = Fill_with_col('*1', ['SN1','DES1'], df_mmsta)

# df_mmsta = Fill_with_col('**2', ['SN2','DES2'], df_mmsta)

# df_mmsta = Fill_with_col('***3', ['SN3','DES3'], df_mmsta)

In [322]:
df_mmsta = df_mmsta[['Level' ,'PN','SN1','DES1','SN2','DES2','SN3','DES3', 'Component quantity']]

df_mmsta


,Level,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,*1,403881526S,S009238422,Super Group 001 See drawing,(Blank),(Blank),(Blank),(Blank),1
1,**2,403881526S,S009238422,Super Group 001 See drawing,S009238343,Double Crimping 001 See drawing,(Blank),(Blank),1
2,***3,403881526S,S009238422,Super Group 001 See drawing,S009238343,Double Crimping 001 See drawing,S009238020,Circuit 0258 COFLRYB 5 R,1
3,***3,403881526S,S009238422,Super Group 001 See drawing,S009238343,Double Crimping 001 See drawing,S009238019,Circuit 0248 COFLRYB 3 R,1
4,**2,403881526S,S009238422,Super Group 001 See drawing,S009238342,Joint 0618 See drawing,S009238019,Circuit 0248 COFLRYB 3 R,1
...,...,...,...,...,...,...,...,...,...
3335,**2,403885584S,S009238444,Twisted Wire 05 ZZ,S009238433,Circuit 0339 COFLRYB 0.5 W,S008818815,Circuit 0375 COFLRYB 0.5 G,1
3336,*1,403885584S,S009238172,Circuit 0005 COFLRYB 0.35 G/W,S009238433,Circuit 0339 COFLRYB 0.5 W,S008818815,Circuit 0375 COFLRYB 0.5 G,1
3337,*1,403885584S,S009238173,Circuit 0011 COFLRYB 0.35 P/W,S009238433,Circuit 0339 COFLRYB 0.5 W,S008818815,Circuit 0375 COFLRYB 0.5 G,1
3338,*1,403885584S,S009238174,Circuit 0013 COFLRYB 0.35 L/W,S009238433,Circuit 0339 COFLRYB 0.5 W,S008818815,Circuit 0375 COFLRYB 0.5 G,1


In [323]:
df_mmsta_pivot = pd.pivot_table(df_mmsta, values = 'Component quantity', index = ['Level','SN1','DES1','SN2' , 'DES2','SN3', 'DES3'], 
                                columns = 'PN', aggfunc = 'size', fill_value = ' ')
# df_mmsta_pivot = df_mmsta_pivot.reset_index()

df_mmsta_pivot

PN                                                                                                                                   403881526S  \
Level SN1        DES1                           SN2        DES2                            SN3        DES3                                        
***3  S009238422 Super Group 001 See drawing    S009238342 Joint 0618 See drawing          S008819089 Circuit 0249 COFLRYB 3 R                1   
                                                                                           S008819090 Circuit 0260 COFLRYB 5 R                1   
                                                                                           S009238018 Circuit 0271 COFLRYB 5 R                1   
                                                S009238343 Double Crimping 001 See drawing S009238019 Circuit 0248 COFLRYB 3 R                1   
                                                                                           S009238020 Circuit 0258 COFLRYB 5 R                1   
...                                                                                                                                         ...   
*1    S009238446 Super Group 008 See drawing    S008819450 Twisted Wire 22   ZZ            S008818815 Circuit 0375 COFLRYB 0.5 G                  
                                                S009238150 Circuit 0379 A3ZP 0.5 GY        S009238031 Circuit 0371 COFLRYB 0.35 BR/W              
                                                S009238352 Joint 0630 See drawing          S008819094 Circuit 0122 COFLRYB 0.5 B                  
      S009238447 Circuit 0084 COFLRYB 0.35 BE/W S009238150 Circuit 0379 A3ZP 0.5 GY        S009238031 Circuit 0371 COFLRYB 0.35 BR/W              
                                                S009238156 Circuit 0337 A3ZP 0.35 BR/W     S008819089 Circuit 0249 COFLRYB 3 R                    

PN                                                                                                                                   403882047S  \
Level SN1        DES1                           SN2        DES2                            SN3        DES3                                        
***3  S009238422 Super Group 001 See drawing    S009238342 Joint 0618 See drawing          S008819089 Circuit 0249 COFLRYB 3 R                1   
                                                                                           S008819090 Circuit 0260 COFLRYB 5 R                1   
                                                                                           S009238018 Circuit 0271 COFLRYB 5 R                1   
                                                S009238343 Double Crimping 001 See drawing S009238019 Circuit 0248 COFLRYB 3 R                1   
                                                                                           S009238020 Circuit 0258 COFLRYB 5 R                1   
...                                                                                                                                         ...   
*1    S009238446 Super Group 008 See drawing    S008819450 Twisted Wire 22   ZZ            S008818815 Circuit 0375 COFLRYB 0.5 G                  
                                                S009238150 Circuit 0379 A3ZP 0.5 GY        S009238031 Circuit 0371 COFLRYB 0.35 BR/W              
                                                S009238352 Joint 0630 See drawing          S008819094 Circuit 0122 COFLRYB 0.5 B                  
      S009238447 Circuit 0084 COFLRYB 0.35 BE/W S009238150 Circuit 0379 A3ZP 0.5 GY        S009238031 Circuit 0371 COFLRYB 0.35 BR/W              
                                                S009238156 Circuit 0337 A3ZP 0.35 BR/W     S008819089 Circuit 0249 COFLRYB 3 R                    

PN                                                                                                                                   403882377S  \
Level SN1        DES1        

In [324]:
import xlwings as xw

wb = xw.Book()
if len(wb.sheets) == 0:
	sheet_mmsta = wb.sheets.add(name="sep mmsta")
else:
	sheet_mmsta = wb.sheets[0]
	sheet_mmsta.name = "sep mmsta"

In [325]:
df_mmsta_pivot = df_mmsta_pivot.reset_index()

In [326]:
df_mmsta_pivot.loc[df_mmsta_pivot['Level'] == '*1', ['SN2', 'DES2', 'SN3', 'DES3']] = '(blank)'
df_mmsta_pivot.loc[df_mmsta_pivot['Level'] == '**2', ['SN3', 'DES3']] = '(blank)'
df_mmsta_pivot

PN,Level,SN1,DES1,SN2,DES2,SN3,DES3,403881526S,403882047S,403882377S,403885221S,403885584S,403887234S
0,***3,S009238422,Super Group 001 See drawing,S009238342,Joint 0618 See drawing,S008819089,Circuit 0249 COFLRYB 3 R,1,1,1,1,1,1
1,***3,S009238422,Super Group 001 See drawing,S009238342,Joint 0618 See drawing,S008819090,Circuit 0260 COFLRYB 5 R,1,1,1,1,1,1
2,***3,S009238422,Super Group 001 See drawing,S009238342,Joint 0618 See drawing,S009238018,Circuit 0271 COFLRYB 5 R,1,1,1,1,1,1
3,***3,S009238422,Super Group 001 See drawing,S009238343,Double Crimping 001 See drawing,S009238019,Circuit 0248 COFLRYB 3 R,1,1,1,1,1,1
4,***3,S009238422,Super Group 001 See drawing,S009238343,Double Crimping 001 See drawing,S009238020,Circuit 0258 COFLRYB 5 R,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,*1,S009238446,Super Group 008 See drawing,(blank),(blank),(blank),(blank),,,1,,,
2332,*1,S009238446,Super Group 008 See drawing,(blank),(blank),(blank),(blank),,,,,1,
2333,*1,S009238446,Super Group 008 See drawing,(blank),(blank),(blank),(blank),,,,1,,
2334,*1,S009238447,Circuit 0084 COFLRYB 0.35 BE/W,(blank),(blank),(blank),(blank),,,,,1,


In [327]:
def level_sorter(level):
    return int(level[-1])

df_mmsta_pivot['level_sort'] = df_mmsta_pivot['Level'].apply(level_sorter)
df_mmsta_pivot = df_mmsta_pivot.sort_values('level_sort').drop(columns='level_sort')

df_mmsta_pivot

PN,Level,SN1,DES1,SN2,DES2,SN3,DES3,403881526S,403882047S,403882377S,403885221S,403885584S,403887234S
2335,*1,S009238447,Circuit 0084 COFLRYB 0.35 BE/W,(blank),(blank),(blank),(blank),,,,,,1
1584,*1,S009238255,Circuit 0209 COFLRYB 1.5 R/BR,(blank),(blank),(blank),(blank),1,,,,,
1585,*1,S009238256,Circuit 0210 COFLRYB 1.5 R/BR,(blank),(blank),(blank),(blank),,1,,,1,
1586,*1,S009238256,Circuit 0210 COFLRYB 1.5 R/BR,(blank),(blank),(blank),(blank),,,,,,1
1587,*1,S009238256,Circuit 0210 COFLRYB 1.5 R/BR,(blank),(blank),(blank),(blank),,,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,***3,S009238426,Super Group 029 See drawing,S009238352,Joint 0630 See drawing,S009238037,Circuit 0134 A3ZP 0.5 B,1,1,1,1,1,1
29,***3,S009238426,Super Group 029 See drawing,S009238352,Joint 0630 See drawing,S009238038,Circuit 0317 COFLRYB 0.75 B/BR,1,1,1,1,1,1
30,***3,S009238426,Super Group 029 See drawing,S009238353,Double Crimping 006 See drawing,S009238039,Circuit 0221 COFLRYB 1 B,1,1,1,1,1,1
16,***3,S009238424,Super Group 014 See drawing,S009238347,Twisted Wire 10 ZZ,S009238028,Circuit 0352 COFLRYB 0.35 P/W,1,1,1,1,1,1


In [298]:
# Sep_sheet = wb.sheets.add(after=sheet_mmsta)
# Sep_sheet.name = "Sep Sheet"
# sheet_mmsta.range("A1").value = [df_mmsta_pivot.columns.tolist()] + df_mmsta_pivot.values.tolist()

In [328]:
df_mmsta_pivot[df_mmsta_pivot.columns[7:]] = df_mmsta_pivot[df_mmsta_pivot.columns[7:]].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

In [329]:
columns_to_agg = list(df_mmsta_pivot.columns[7:])
cols = ['Level', 'SN1', 'DES1', 'SN2', 'DES2', 'SN3', 'DES3']
cols.extend(columns_to_agg)
cols

['Level',
 'SN1',
 'DES1',
 'SN2',
 'DES2',
 'SN3',
 'DES3',
 '403881526S',
 '403882047S',
 '403882377S',
 '403885221S',
 '403885584S',
 '403887234S']

In [330]:
agg_dict = {col: 'sum' for col in columns_to_agg}

df_mmsta_pivot = df_mmsta_pivot.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

df_mmsta_pivot = df_mmsta_pivot[cols]
df_mmsta_pivot.replace(to_replace=" " , value="(Blank)")
df_mmsta_pivot.replace(to_replace= 0 , value=" ")


sheet_mmsta.range("A1").value = [df_mmsta_pivot.columns.tolist()] + df_mmsta_pivot.values.tolist()

In [331]:
df_fil_simple = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'circuit' in x.lower())]
columns_to_agg = list(df_fil_simple.columns[7:])

agg_dict = {col: 'sum' for col in columns_to_agg}

df_fil_simple = df_fil_simple.groupby(['Level','SN1']).agg({
    'DES1': 'first',
    'SN2': 'first',
    'DES2': 'first',
    'SN3': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

df_fil_simple.replace(to_replace=" " , value="(Blank)")

fil_simple_sheet = wb.sheets.add(after=sheet_mmsta)
fil_simple_sheet.name = "Fil Simple"
fil_simple_sheet.range("A1").value = [df_fil_simple.columns.tolist()] + df_fil_simple.values.tolist()

In [332]:
df_double = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'double' in x.lower())]
df_double = df_double.groupby(['Level','SN1']).agg({
    'DES1': 'first',
    'SN2': 'first',
    'DES2': 'first',
    'SN3': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

double_sheet = wb.sheets.add(after=fil_simple_sheet)
double_sheet.name = "Double"
double_sheet.range("A1").value = [df_double.columns.tolist()] + df_double.values.tolist()

In [333]:
df_twisted = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'twisted' in x.lower())]
df_twisted = df_twisted.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
twisted_sheet = wb.sheets.add(after=double_sheet)
twisted_sheet.name = "Twisted"
twisted_sheet.range("A1").value = [df_twisted.columns.tolist()] + df_twisted.values.tolist()

In [334]:
df_joint = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'joint' in x.lower())]
df_joint = df_joint.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
joint_sheet = wb.sheets.add(after=twisted_sheet)
joint_sheet.name = "joint"
joint_sheet.range("A1").value = [df_joint.columns.tolist()] + df_joint.values.tolist()

In [335]:
df_super_grp = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'super group' in x.lower())]
df_super_grp = df_super_grp.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
superGrp_sheet = wb.sheets.add(after=joint_sheet)
superGrp_sheet.name = "Super group"
superGrp_sheet.range("A1").value = [df_super_grp.columns.tolist()] + df_super_grp.values.tolist()

In [336]:
df_2 = df_super_grp[df_super_grp['SN2'] != '(blank)']
shape_super_grp = df_2[df_2['SN3'] != '(blank)'].shape[0]

shape_joint = df_joint[df_joint['SN2'] != '(blank)'].shape[0]

shape_twisted = df_twisted[df_twisted['SN2'] != '(blank)'].shape[0]

df_double_filtred = df_mmsta_pivot[df_mmsta_pivot['DES1'].apply(lambda x : 'double' in x.lower())]
shape_double = df_double_filtred[df_double_filtred['SN2'] != '(blank)'].shape[0]

shape_simple = df_fil_simple.shape[0]


#to excel in another sheet
summary = wb.sheets.add(after=superGrp_sheet)
summary.name = "Summary"
summary.range("A1").value = 'TYPE'
summary.range("B1").value = 'Count df_mmsta'
summary.range("A2").value = 'Circuit'
summary.range("B2").value = df_mmsta_pivot[df_mmsta_pivot['DES1'].apply(lambda x : 'circuit' in x.lower())].shape[0]
summary.range("A3").value = 'Double'
summary.range("B3").value = shape_double
summary.range("A4").value = 'Twisted'
summary.range("B4").value = shape_twisted
summary.range("A5").value = 'Joint'
summary.range("B5").value = shape_joint
summary.range("A6").value = 'Super group'
summary.range("B6").value = shape_super_grp

In [308]:
shape_joint

82

In [309]:
df_joint = df_mmsta_pivot[df_mmsta_pivot['DES1'].apply(lambda x : 'joint' in x.lower())]
shape_joint = df_joint[df_joint['SN2'] != '(blank)'].shape[0]
shape_joint

82

In [310]:
end_time = time.time()

execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time / 60} minutes")

Temps d'exécution : 0.3830645521481832 minutes
